In [1]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
!pip install keras-tcn
from tcn import TCN

     |████████████████████████████████| 1.1 MB 4.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset1/acc+gyr/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testX.npy')
y_test = np.load('gdrive/My Drive/dataset1/acc+gyr/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [6]:
from tensorflow import keras
input_shape = x_train.shape[1:]
inp = Input(shape=x_train.shape[1:])
x = SpatialDropout1D(0.2)(inp)
x = TCN(128,dilations = [1, 2, 4], return_sequences=True)(x)
x = TCN(64,dilations = [1, 2, 4], return_sequences=True)(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
    
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.2)(conc)
outp = Dense(118, activation="softmax")(conc)    
model = Model(inputs=inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 6)]     0           []                               
                                                                                                  
 spatial_dropout1d_2 (SpatialDr  (None, 128, 6)      0           ['input_3[0][0]']                
 opout1D)                                                                                         
                                                                                                  
 tcn_4 (TCN)                    (None, 128, 128)     249728      ['spatial_dropout1d_2[0][0]']    
                                                                                                  
 tcn_5 (TCN)                    (None, 128, 64)      94656       ['tcn_4[0][0]']            

In [9]:
history = model.fit(x_train, y_train, epochs=150, verbose=True, validation_data=(x_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/150
207/207 [==============================] - 32s 91ms/step - loss: 4.0731 - accuracy: 0.1350 - val_loss: 2.0684 - val_accuracy: 0.5233
Epoch 2/150
207/207 [==============================] - 16s 77ms/step - loss: 2.0906 - accuracy: 0.4566 - val_loss: 0.8337 - val_accuracy: 0.8156
Epoch 3/150
207/207 [==============================] - 16s 77ms/step - loss: 1.3613 - accuracy: 0.6348 - val_loss: 0.4862 - val_accuracy: 0.8894
Epoch 4/150
207/207 [==============================] - 16s 77ms/step - loss: 1.0436 - accuracy: 0.7188 - val_loss: 0.3595 - val_accuracy: 0.9104
Epoch 5/150
207/207 [==============================] - 16s 76ms/step - loss: 0.8533 - accuracy: 0.7688 - val_loss: 0.3182 - val_accuracy: 0.9234
Epoch 6/150
207/207 [==============================] - 16s 76ms/step - loss: 0.7216 - accuracy: 0.7992 - val_loss: 0.2810 - val_accuracy: 0.9243
Epoch 7/150
207/207 [==============================] - 16s 76ms/step - loss: 0.6486 - accuracy: 0.8200 - val_loss: 0.2392 - val_ac

In [10]:
loss, accuracy = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9736
Testing Accuracy: 0.9206
